# Function calling
#### **General idea**
Let LLM know how to prepare its output so it can be used by your functions


#### **Explanation**
LLM can't execute any functions by itself, but if its object is initialized in right way it may
1. Get list of functions that may be executed with his output
2. Get list of desired input for each function
3. Choose which function from your list is best fit to your prompt
4. Prepare its result in json fromat that may be passed to function later

# Lets start with some smple function

In [1]:
def count_elephants(some_str, case_sensitive=False, output_format="num"):
    """
    It counts ELEPHANTS in given text
    INPUTS:
    some_str: str - text that we use to count elephants
    case_sensitive: bool - 
    output_format: str - ["num", "text]" - to present implementation enum in function_calling
    """
    if not case_sensitive:
        some_str = some_str.lower()
    cnt = len(some_str.split("elephant"))-1
    if output_format == "num": 
        return cnt
    else:
        return f"Number of elephants is {cnt}"

count_elephants(
    "In this text we have John - The BIG ELEPHANT, and Susan, the little elephant", 
    case_sensitive=True, output_format="text"
    )

'Number of elephants is 1'

## Preparing function schema and passing it to LLM

#### 1. Using python dictionaries

In [2]:
count_elephants_schema = {
    "name": "count_elephants",
    "description": "Counts elephants in some text provided by user",
    "parameters": {
        "type": "object",
        "properties": {
            "some_str": {
                "type": "array",
                "description": "Extracted part of user message that contains text to count elephants",
                "items": {
                    "type": "string"
                }
            },
            "case_sensitive": {
                "type": "boolean",
                "description": "True if user specifies he want to counts only SMALL or BIG elephants or if he expectly mentiond it should be case sensitive"
            },
            "output_format": {
                "type": "string",
                "description": "num (if user ask for counting elephants) text (if user ask to tell him how many elephants)",
                "enum": ["num", "text"]
            }
        },
        "required": [
            "type", "tags", "command"
        ]
    }
}

**initialization model with funcitons**

In [3]:
# --------------------------------------------------------------
# WITHOUT FUNCTION_CALL SPECIFIED (which indicates default function)
# --------------------------------------------------------------
# Model will respond normally (without functions) unless function use would be obvvious
from langchain_openai import ChatOpenAI
from langchain.schema import BaseMessage, HumanMessage, AIMessage

model_with_functions = ChatOpenAI(
    model_name="gpt-4-0613",
    model_kwargs={
        "functions": [count_elephants_schema]
        # "function_call": {"name": "query_enrichment"}, 
    }
)


**model_with_functions response for non-related to elephants stuff (no function_call specified)**

In [4]:
import json
result = model_with_functions.invoke([HumanMessage(content="Hey there!")])
print(json.dumps(json.loads(result.json()), indent=4))

{
    "content": "Hello! How can I assist you today?",
    "additional_kwargs": {},
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 10,
            "prompt_tokens": 131,
            "total_tokens": 141
        },
        "model_name": "gpt-4-0613",
        "system_fingerprint": null,
        "finish_reason": "stop",
        "logprobs": null
    },
    "type": "ai",
    "name": null,
    "id": "run-cf74bc71-ca8a-4723-929a-abcca481301f-0",
    "example": false
}


**model_with_functions response for asking to count elephants (no function_call specified)**

In [5]:
import json
result = model_with_functions.invoke(
    [
        HumanMessage(content='Please, count elephants in the text "Elephants? I love elephants! Elephants are fucking amazing!')
    ]
)
print(json.dumps(json.loads(result.json()), indent=4))

{
    "content": "",
    "additional_kwargs": {
        "function_call": {
            "arguments": "{\n  \"some_str\": [\"Elephants? I love elephants! Elephants are fucking amazing!\"],\n  \"output_format\": \"text\"\n}",
            "name": "count_elephants"
        }
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 37,
            "prompt_tokens": 149,
            "total_tokens": 186
        },
        "model_name": "gpt-4-0613",
        "system_fingerprint": null,
        "finish_reason": "function_call",
        "logprobs": null
    },
    "type": "ai",
    "name": null,
    "id": "run-4d639074-52bd-4db7-a157-37611146db34-0",
    "example": false
}


**model_with_functions response for non-related to elephants stuff (WITH function_call specified)**

In [13]:
# --------------------------------------------------------------
# WITH FUNCTION_CALL SPECIFIED (default = count_elaphants)
# --------------------------------------------------------------
# (model will choose this if context not suggest any other from list)
from langchain_openai import ChatOpenAI

model_with_functions = ChatOpenAI(
    model_name="gpt-4-0613",
    model_kwargs={
        "functions": [count_elephants_schema],
        "function_call": {"name": "count_elephants"}, 
    }
)

import json
result = model_with_functions.invoke([HumanMessage(content="Hey there!")])
print(json.dumps(json.loads(result.json()), indent=4))

{
    "content": "",
    "additional_kwargs": {
        "function_call": {
            "arguments": "{\n  \"some_str\": [\"Hey there!\"]\n}",
            "name": "count_elephants"
        }
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 12,
            "prompt_tokens": 138,
            "total_tokens": 150
        },
        "model_name": "gpt-4-0613",
        "system_fingerprint": null,
        "finish_reason": "stop",
        "logprobs": null
    },
    "type": "ai",
    "name": null,
    "id": "run-0b9dd49b-edb8-498c-a024-bd6693887832-0",
    "example": false
}


#### 2. Using python BaseModel and pydantic

**Defining pydantic BaseModel to let LLM know about structure**

```
NOTE: There is STR value in output_format instead of ENUM. 
Using ENUM here requires Enum library and defining new class
```

In [7]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from pydantic import BaseModel, Field

class CountElephantsInput(BaseModel):
    some_str: str = Field(..., description="Extracted part of user message that contains text to count elephants")
    case_sensitive: bool = Field(False, description="True if user specifies he want to counts only SMALL or BIG elephants or if he expectly mentiond it should be case sensitive")
    output_format: str = Field("num", description="num (if user ask for counting elephants) text (if user ask to tell him how many elephants)")

    class Config:
        schema_extra = {
            "example": {
                "some_str": "There are 5 elephants in the room, and 2 more elephants just entered.",
                "case_sensitive": False,
                "output_format": "text"
            }
        }

**Initialize model with functions schema defined with pydantic**

In [14]:
# Initialize ChatOpenAI with the bound function
model = ChatOpenAI(
    model_name="gpt-4-0613",
    temperature=0
)

model_with_functions_pydantic = model.bind_functions(
    functions=[CountElephantsInput],
    function_call={"name": "CountElephantsInput"}
)

**model_with_functions_pydantic**

In [56]:
user_input = "Hey there! Lets count elephants in just single example: 'this is elephant, this is another elephant so we have two of them, right?'"
result = model_with_functions_pydantic.invoke([HumanMessage(content=user_input)])
print(json.dumps(json.loads(result.json()), indent=4))

{
    "content": "",
    "additional_kwargs": {
        "function_call": {
            "arguments": "{\n  \"some_str\": \"this is elephant, this is another elephant so we have two of them, right?\"\n}",
            "name": "CountElephantsInput"
        }
    },
    "response_metadata": {
        "token_usage": {
            "completion_tokens": 25,
            "prompt_tokens": 161,
            "total_tokens": 186
        },
        "model_name": "gpt-4-0613",
        "system_fingerprint": null,
        "finish_reason": "stop",
        "logprobs": null
    },
    "type": "ai",
    "name": null,
    "id": "run-c830ca64-69ab-4ce5-ade5-b228d3c3ca74-0",
    "example": false
}


#### PROS OF PYDANTIC - we can easly test model outpus against defined class

In [57]:
# Lets take previous example function args
function_name = result.additional_kwargs["function_call"]["name"]
print(function_name)

function_args = result.additional_kwargs["function_call"]["arguments"]
print(function_args)

# Lets validate&parse it
validated_args = CountElephantsInput.parse_raw(function_args)
validated_args

CountElephantsInput
{
  "some_str": "this is elephant, this is another elephant so we have two of them, right?"
}


CountElephantsInput(some_str='this is elephant, this is another elephant so we have two of them, right?', case_sensitive=False, output_format='num')

In [58]:
non_valid_args ='{"some_str": "Hey there!", "case_sensitive": "it should be boolean"}'
print(non_valid_args)

# Lets validate&parse it
validated_args = CountElephantsInput.parse_raw(non_valid_args)
validated_args

{"some_str": "Hey there!", "case_sensitive": "it should be boolean"}


ValidationError: 1 validation error for CountElephantsInput
case_sensitive
  value could not be parsed to a boolean (type=type_error.bool)

# Example of executing

In [63]:
# Define list of functions available to model
function_mapping = {'CountElephantsInput':count_elephants}

# Our function have not defined validation inside it so lets assume we validate args before passing it
# For this example i will pick values from Pydantic validation example
function_name = function_name
function_kwargs = validated_args.dict()
print(function_name, json.dumps(function_kwargs,indent=4))

# Execute function
print("\n\nResult:")
function_mapping[function_name](**function_kwargs)

CountElephantsInput {
    "some_str": "this is elephant, this is another elephant so we have two of them, right?",
    "case_sensitive": false,
    "output_format": "num"
}


Result:


2